In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import json
import os

In [2]:
14%5

4

In [2]:
from utils import * 
from dataloader import *
from txlstm_szpool import *

/home/deeksha/anaconda3/envs/deepai/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
np.random.RandomState()

RandomState(MT19937) at 0x7FD7EC6175A0

In [6]:
pt_list = np.load('pts.npy')
manifest = read_manifest('data/tuh_single_windowed_manifest.csv', ',')
np.random.shuffle(pt_list)



In [ ]:
np.save('pts_rs1.npy', pt_list)
del pt_list
pt_list_rs0 = np.load('pts_rs1.npy')

In [7]:
nfolds = 5 
testsize = 24
step = 24
for fold in range(nfolds):
    test_pts = pt_list_rs0[fold*step: (fold+1)*step]
    train_pts = [pt for pt in pt_list_rs0 if pt not in test_pts ]
    
    if not os.path.exists('crossval_rs1'):
        os.mkdir('crossval_rs1')
    if not os.path.exists('crossval_rs1/fold'+str(fold)):
        os.mkdir('crossval_rs1/fold'+str(fold))
        
    loc = 'crossval_rs1/fold'+str(fold)+'/'
    np.save(loc+'pts_test'+str(fold)+'.npy', test_pts)
    np.save(loc+'pts_train'+str(fold)+'.npy', train_pts)

In [22]:
for fold in range(nfolds):
    pts_old = np.load('crossval/fold'+str(fold)+'/pts_test'+str(fold)+'.npy')
    pts_rs0 = np.load('crossval_rs0/fold'+str(fold)+'/pts_test'+str(fold)+'.npy')
    pts_rs1 = np.load('crossval_rs1/fold'+str(fold)+'/pts_test'+str(fold)+'.npy')
    pts_old.sort()
    pts_rs0.sort()
    pts_rs1.sort()
    if (pts_old == pts_rs0).all() or (pts_rs0 == pts_rs1).all() or (pts_old == pts_rs1).all():
        print('oops')
    else:
        print('yay')

yay
yay
yay
yay
yay


In [18]:
pr

In [6]:
for fold in range(5):
    test = np.load('crossval_rs0/fold'+str(fold)+'/pts_test'+str(fold)+'.npy')
    train = np.load('crossval_rs0/fold'+str(fold)+'/pts_train'+str(fold)+'.npy')
    trig = 0
    for t in train:
        if t in test:
            trig = 1
    if trig == 0:
        print(fold, 'safe')
    else:
        print(fold, 'oops')

0 safe
1 safe
2 safe
3 safe
4 safe


# dataset


In [7]:
pt_list = np.load('pts.npy')
manifest = read_manifest('data/tuh_single_windowed_manifest.csv', ',')

In [8]:
len(manifest)

634

In [9]:
len(pt_list)

124

In [10]:
manifest[0]

OrderedDict([('', '0'),
             ('pt_id', '883'),
             ('fn', '00000883_s002_t000.npy'),
             ('loc', 'data/tuh_windowed/00000883_s002_t000_win.npy'),
             ('fs', '200'),
             ('duration', '600'),
             ('nsamples', '320200'),
             ('nchns', '19'),
             ('nsz', '10'),
             ('sz_starts',
              '[20.636, 181.14, 337.1, 489.884, 643.744, 801.684, 944.012, 1118.98, 1273.7128, 1420.072]'),
             ('sz_ends',
              '[115.516, 277.1, 423.828, 576.86, 733.358, 876.0815, 1026.2029, 1212.088, 1396.068, 1521.948]'),
             ('onset_zone', '[2]'),
             ('hemi', 'R'),
             ('region', 'P'),
             ('fp1', '0'),
             ('f7', '0'),
             ('t3', '0'),
             ('t5', '0'),
             ('o1', '0'),
             ('f3', '0'),
             ('c3', '0'),
             ('p3', '0'),
             ('fz', '0'),
             ('cz', '0'),
             ('pz', '0'),
             ('fp2

# nsz

In [13]:
pt_sz = dict(zip(pt_list, [0]*len(pt_list)))

for mn in manifest:
    ptid = int(mn['pt_id'])
    val = pt_sz[ptid]
    n = min(int(mn['nsz']), 10)
    #n = min(n, 10)
#
    pt_sz[ptid] = val+n

In [14]:
nseiz = np.array(list(pt_sz.values()))
print(nseiz.mean(),nseiz.std(), nseiz.max(), nseiz.min())

12.459677419354838 20.346751818739055 146 1


# dur

In [15]:
oldman = read_manifest('data/tuh_windowed_manifest.csv', ',')

In [16]:
oldman[9]

OrderedDict([('', '11'),
             ('pt_id', '2806'),
             ('fn', '00002806_s001_t003.npy'),
             ('loc', 'data/tuh_preprocessed/00002806_s001_t003_win.npy'),
             ('fs', '200'),
             ('duration', '165'),
             ('nsamples', '33000'),
             ('nchns', '19'),
             ('nsz', '3'),
             ('sz_starts', '[1.0, 52.3775, 149.715]'),
             ('sz_ends', '[17.35, 111.2374, 163.715]'),
             ('onset_zone', '[2]'),
             ('hemi', 'R'),
             ('region', 'P'),
             ('fp1', '0'),
             ('f7', '0'),
             ('t3', '0'),
             ('t5', '0'),
             ('o1', '0'),
             ('f3', '0'),
             ('c3', '0'),
             ('p3', '0'),
             ('fz', '0'),
             ('cz', '0'),
             ('pz', '0'),
             ('fp2', '0'),
             ('f8', '1'),
             ('t4', '1'),
             ('t6', '0'),
             ('o2', '0'),
             ('f4', '0'),
             ('c4'

In [17]:
len(oldman)

641

In [18]:
manifest[0]

OrderedDict([('', '0'),
             ('pt_id', '883'),
             ('fn', '00000883_s002_t000.npy'),
             ('loc', 'data/tuh_windowed/00000883_s002_t000_win.npy'),
             ('fs', '200'),
             ('duration', '600'),
             ('nsamples', '320200'),
             ('nchns', '19'),
             ('nsz', '10'),
             ('sz_starts',
              '[20.636, 181.14, 337.1, 489.884, 643.744, 801.684, 944.012, 1118.98, 1273.7128, 1420.072]'),
             ('sz_ends',
              '[115.516, 277.1, 423.828, 576.86, 733.358, 876.0815, 1026.2029, 1212.088, 1396.068, 1521.948]'),
             ('onset_zone', '[2]'),
             ('hemi', 'R'),
             ('region', 'P'),
             ('fp1', '0'),
             ('f7', '0'),
             ('t3', '0'),
             ('t5', '0'),
             ('o1', '0'),
             ('f3', '0'),
             ('c3', '0'),
             ('p3', '0'),
             ('fz', '0'),
             ('cz', '0'),
             ('pz', '0'),
             ('fp2

In [20]:
pt_totdur = dict(zip(pt_list, [0]*len(pt_list)))
pt_szdur = dict(zip(pt_list, [[]]*len(pt_list)))
for i, mn in enumerate(oldman):
    
    ptid = int(mn['pt_id'])
    
    dur = pt_totdur[ptid]
    szdur = list(np.copy(pt_szdur[ptid]))
  
    val = int(mn['duration'])
 
    pt_totdur[ptid] = dur+val
    
    s = np.array(json.loads(mn['sz_starts']))
    e = np.array(json.loads(mn['sz_ends']))
    vals = np.round(e-s)[:10]
   
    szdur.append(vals)
    pt_szdur[ptid] = np.copy(szdur)

    

/home/deeksha/anaconda3/envs/deepai/lib/python3.7/site-packages/numpy/lib/function_base.py:804: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, order=order, subok=subok, copy=True)


In [21]:
durs = np.array(list(pt_totdur.values()))/3600
print(durs.mean(),durs.std(), durs.max(), durs.min())

1.315013440860215 2.205739717234951 20.94333333333333 0.0025


In [22]:
szdurs = []
for i in pt_list:
    szdurs.append(np.concatenate(pt_szdur[i]).mean())
    
szdurs = np.array(szdurs)

In [23]:
print(szdurs.mean(),szdurs.std(), szdurs.max(), szdurs.min())

88.98503241601 121.479206821697 1121.0 7.5


In [122]:
len(durs)

124

In [128]:
df = pd.DataFrame({'pt id': list(pt_sz.keys()),
                    'nsz': list(pt_sz.values()), 
                 'tot dur': durs, 
                 'sz dur': szdurs})

In [130]:
df.to_csv('demo.csv')


In [155]:
lr = []
ap = []
qd = []
c = []
for pt in pt_list:
    for mn in oldman:
        if int(mn['pt_id']) == pt:
            break
    
    if mn['hemi'][0] == 'R' or mn['hemi'][0] == 'r':
        lr.append(1)
        h = 1
    else:
        lr.append(0)
        h = 0
    if mn['region'][0] == 'A' or mn['region'][0] == 'a':
        ap.append(0)
        r = 0
    else:
        ap.append(1)
        r = 1
    print(pt, mn['pt_id'], h)    
    if h==0 and r==0:
        qd.append(0)
    elif h ==1 and r==0:
        qd.append(1)
    elif h ==1 and r==1:
        qd.append(2)
    else:
        qd.append(3)
    c.append(mn['Comments'])

883 883 1
906 906 1
1981 1981 0
2521 2521 1
2707 2707 0
2806 2806 1
2991 2991 1
4049 4049 1
4456 4456 1
5427 5427 1
6103 6103 0
6175 6175 0
6440 6440 1
6444 6444 1
6455 6455 0
6535 6535 0
6563 6563 1
6811 6811 0
6904 6904 0
7032 7032 1
7095 7095 0
7235 7235 0
7446 7446 0
7584 7584 0
7793 7793 1
7839 7839 1
7936 7936 0
7937 7937 0
8092 8092 1
8164 8164 0
8295 8295 1
8345 8345 1
8444 8444 0
8476 8476 0
8527 8527 0
8608 8608 0
8616 8616 1
9050 9050 1
9104 9104 0
9158 9158 0
9231 9231 1
9232 9232 1
9370 9370 1
9540 9540 0
9623 9623 0
9630 9630 1
9842 9842 1
9885 9885 1
9934 9934 1
10088 10088 0
10158 10158 0
10418 10418 0
10489 10489 1
10551 10551 1
10587 10587 1
10591 10591 1
10760 10760 0
10843 10843 0
11077 11077 0
11272 11272 1
11321 11321 0
11333 11333 0
11562 11562 1
11869 11869 1
11870 11870 0
11873 11873 0
11914 11914 1
11915 11915 1
11927 11927 0
11972 11972 1
12409 12409 1
12484 12484 1
12679 12679 1
12759 12759 1
12966 12966 1
12973 12973 0
13095 13095 1
13145 13145 0
13407 1340

In [156]:
df = pd.DataFrame({'pt id': list(pt_sz.keys()),
                    'nsz': list(pt_sz.values()), 
                 'tot dur': durs, 
                 'sz dur': szdurs, 
                  'hemi': lr, 
                  'region': ap, 
                  'quad':qd,
                   'comments':c 
                  })

In [157]:
df.to_csv('demo.csv')


In [142]:
manifest[0]

OrderedDict([('', '0'),
             ('pt_id', '883'),
             ('fn', '00000883_s002_t000.npy'),
             ('loc', 'data/tuh_windowed/00000883_s002_t000_win.npy'),
             ('fs', '200'),
             ('duration', '600'),
             ('nsamples', '320200'),
             ('nchns', '19'),
             ('nsz', '10'),
             ('sz_starts',
              '[20.636, 181.14, 337.1, 489.884, 643.744, 801.684, 944.012, 1118.98, 1273.7128, 1420.072]'),
             ('sz_ends',
              '[115.516, 277.1, 423.828, 576.86, 733.358, 876.0815, 1026.2029, 1212.088, 1396.068, 1521.948]'),
             ('onset_zone', '[2]'),
             ('hemi', 'R'),
             ('region', 'P'),
             ('fp1', '0'),
             ('f7', '0'),
             ('t3', '0'),
             ('t5', '0'),
             ('o1', '0'),
             ('f3', '0'),
             ('c3', '0'),
             ('p3', '0'),
             ('fz', '0'),
             ('cz', '0'),
             ('pz', '0'),
             ('fp2

In [158]:
120-66

54

In [13]:
pt1 = np.load('data/lastminute/final_models/fold5/pts_train0.npy')

In [16]:
pt2 = np.load('data/lastminute/crossval_rs0/fold0/pts_train0.npy')

In [17]:
pt1 == pt2

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])